In [2]:
#import modules
import pandas as pd
import numpy as np
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LassoCV
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.feature_selection import mutual_info_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import mean_squared_error

upload 'shhs1-dataset-0.20.0 (1).csv' and 'shhs-data-dictionary-0.20.0-variables.csv'

In [4]:
#read Sleep Heart Health Study Dataset

shhs1 = pd.read_csv('/shhs1-dataset-0.20.0 (1).csv', error_bad_lines=False)
#read Sleep Heart Health Study Dictionary
var_dict = pd.read_csv('/shhs-data-dictionary-0.20.0-variables.csv')

<ipython-input-4-870f2d3f43b7>:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  shhs1 = pd.read_csv('/shhs1-dataset-0.20.0 (1).csv', error_bad_lines=False)
<ipython-input-4-870f2d3f43b7>:3: DtypeWarning: Columns (1214) have mixed types. Specify dtype option on import or set low_memory=False.
  shhs1 = pd.read_csv('/shhs1-dataset-0.20.0 (1).csv', error_bad_lines=False)


In [5]:
#find the shape of shhs1 - 5804 rows, 1271 columns
shhs1.shape

(5804, 1271)

In [6]:
#find the shape of var_dict - 1896 rows, 11 columns
var_dict.shape

(1896, 11)

# Data Processing

In [7]:
# remove sleep monitoring columns from independent variables (unattinable for predicting)
sleep_monitoring_col = var_dict[var_dict['folder'].str.contains(r'sleep monitoring', case=False, na=False)]['id']

In [8]:
sleep_monitoring_col

633        ahiov50
634     headbox_id
635        hrov150
636        hrund30
637     monitor_id
           ...    
1721      timest1p
1722       timest2
1723      timest2p
1724      timest34
1725          waso
Name: id, Length: 1093, dtype: object

In [9]:
# drop target variables, pptidr (has numerical and categorical values in one column), and sleep monitoring columns in independent variable
x = shhs1.drop(columns=['ahi_c0h3a', 'pptidr'])
for col in sleep_monitoring_col:
    if col in x.columns:
        x = x.drop(columns=col)

y = shhs1['ahi_c0h3a']

In [10]:
x

,nsrrid,pptid,ecgdate,lvh3_1,lvh3_3,st4_1_3,st5_1_3,lvhst,mob1,part2deg,...,educat,date02,date25,weight,waist,height,weight20,lang15,age_category_s1,visitnumber
0,200001,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,-48.0,0.0,65.0,86.0,178.0,69.0,1.0,7,1
1,200002,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,-111.0,-37.0,93.0,107.0,168.0,93.0,NaN,9,1
2,200003,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,-7.0,0.0,51.0,82.0,145.0,50.7,1.0,9,1
3,200004,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,-24.0,0.0,64.0,85.0,180.0,65.4,1.0,6,1
4,200005,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,-89.0,-33.0,56.0,76.0,155.0,56.0,NaN,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5799,205800,5835,-854.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,-168.0,1.0,70.5,99.0,166.0,70.8,1.0,8,1
5800,205801,5836,-755.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,-95.0,1.0,83.6,99.0,176.0,82.8,1.0,6,1
5801,205802,5837,-768.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,-83.0,1.0,75.0,91.0,175.0,74.2,1.0,7,1
5802,205803,5838,-755.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,-89.0,0.0,76.8,93.0,176.0,78.6,1.0,6,1


In [11]:
# normalize x
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

imputer = KNNImputer(n_neighbors=5)  # You can change the number of neighbors if needed
x_imputed_scaled = imputer.fit_transform(x_scaled)

# reverse scaling
x_imputed = scaler.inverse_transform(x_imputed_scaled)

# Convert the result back to DataFrame
x_imputed = pd.DataFrame(x_imputed, columns=x.columns)

problem with knn: how many neighbors are the best?
NaNs categorical variables will be imputed, but the result might not be intepretable

In [12]:
x_imputed

,nsrrid,pptid,ecgdate,lvh3_1,lvh3_3,st4_1_3,st5_1_3,lvhst,mob1,part2deg,...,educat,date02,date25,weight,waist,height,weight20,lang15,age_category_s1,visitnumber
0,200001.0,1.0,-748.2,0.4,0.6,0.2,2.000000e-01,0.2,0.0,0.0,...,3.0,-48.0,0.0,65.0,86.0,178.0,69.0,1.0,7.0,1.0
1,200002.0,2.0,-75.8,0.2,0.2,0.0,2.000000e-01,0.0,0.0,0.0,...,2.0,-111.0,-37.0,93.0,107.0,168.0,93.0,1.0,9.0,1.0
2,200003.0,3.0,-1071.8,0.2,0.0,0.0,4.000000e-01,0.0,0.0,0.0,...,3.0,-7.0,0.0,51.0,82.0,145.0,50.7,1.0,9.0,1.0
3,200004.0,4.0,-728.6,0.0,0.0,0.0,2.000000e-01,0.0,0.0,0.0,...,3.0,-24.0,0.0,64.0,85.0,180.0,65.4,1.0,6.0,1.0
4,200005.0,5.0,-241.8,0.2,0.0,0.0,2.775558e-17,0.0,0.0,0.0,...,2.0,-89.0,-33.0,56.0,76.0,155.0,56.0,1.0,8.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5799,205800.0,5835.0,-854.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,...,2.0,-168.0,1.0,70.5,99.0,166.0,70.8,1.0,8.0,1.0
5800,205801.0,5836.0,-755.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,...,3.0,-95.0,1.0,83.6,99.0,176.0,82.8,1.0,6.0,1.0
5801,205802.0,5837.0,-768.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,...,4.0,-83.0,1.0,75.0,91.0,175.0,74.2,1.0,7.0,1.0
5802,205803.0,5838.0,-755.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,...,2.0,-89.0,0.0,76.8,93.0,176.0,78.6,1.0,6.0,1.0


In [18]:
def find_important_features(model, model_name, target_var):
  model_importances = model.feature_importances_
  model_features = np.array(x_imputed.columns)[model_importances > 0]
  model_dataset = x_imputed[model_features]
  model_dataset[target_var] = shhs1[target_var]
  print(len(model_features))
  csv_name = model_name + "_" + target_var + ".csv"
  model_dataset.to_csv(csv_name, index=False)



# Feature selection

## Target variable: ahi_c0h3a

### Decision Tree


In [13]:
#create decision tree
tree = DecisionTreeRegressor()
tree.fit(x_imputed, y)

DecisionTreeRegressor()

In [19]:
find_important_features(tree, "decision_tree", "ahi_c0h3a")

<ipython-input-18-cfb83178e5c9>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_dataset[target_var] = shhs1[target_var]


256


In [14]:
#identify the most important features through decision trees
tree_importances = tree.feature_importances_
decision_tree_features = np.array(x_imputed.columns)[tree_importances > 0]
decision_tree_dataset = x_imputed[decision_tree_features]
decision_tree_dataset['ahi_c0h3a'] = shhs1['ahi_c0h3a']

<ipython-input-14-c5bced64f58f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  decision_tree_dataset['ahi_c0h3a'] = shhs1['ahi_c0h3a']


In [15]:
#end up with 249 features
len(decision_tree_features)

256

In [16]:
#write to csv
decision_tree_dataset.to_csv('decision_tree_ahi_c0h3a.csv', index=False)

### Random Forest

In [20]:
#create a random forest with 100 "trees"
forest = RandomForestRegressor(n_estimators=100)  # You can change the number of trees if needed
forest.fit(x_imputed, y)

RandomForestRegressor()

In [23]:
find_important_features(forest, "random_forest", "ahi_c0h3a")

<ipython-input-18-cfb83178e5c9>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_dataset[target_var] = shhs1[target_var]


277


In [21]:
#identify the most important features through random forest
forest_importances = forest.feature_importances_
random_forest_features = np.array(x_imputed.columns)[forest_importances > 0]
random_forest_dataset = x_imputed[decision_tree_features]
random_forest_dataset['ahi_c0h3a'] = shhs1['ahi_c0h3a']

<ipython-input-21-d6133958967c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  random_forest_dataset['ahi_c0h3a'] = shhs1['ahi_c0h3a']


In [22]:
#end up with 278 features
len(random_forest_features)

277

In [ ]:
#write to csv
random_forest_dataset.to_csv('random_forest_ahi_c0h3a.csv', index=False)

### Mutual Information

In [ ]:
#create mutual information model
mi = mutual_info_regression(x_imputed, y)

In [ ]:
# Get feature names where mutual information is greater than 0
mi_features = np.array(x_imputed.columns)[mi > 0]

# Create a new dataset with only the features having non-zero mutual information
mi_dataset = x_imputed[mi_features]

# Add the 'ahi_c0h3a' column to the new dataset
mi_dataset['ahi_c0h3a'] = shhs1['ahi_c0h3a']

<ipython-input-20-fb481aa3287c>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mi_dataset['ahi_c0h3a'] = shhs1['ahi_c0h3a']


In [ ]:
#end up with 198 features
len(mi_features)

198

In [ ]:
#write to csv
mi_dataset.to_csv('mi_ahi_c0h3a.csv', index=False)

### Forward selection

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

forward_selection_features = []

# Start with an arbitrarily large initial MSE
previous_mse = float('inf')
mse_change_threshold = 1e-5  # Adjust this threshold value based on the scale of your MSE

#run forward selection
while len(forward_selection_features) < len(x_imputed.columns):
    remaining_features = list(set(x_imputed.columns) - set(forward_selection_features))
    best_feature_mse = float('inf')
    best_feature = None

    for feature in remaining_features:
        candidate_features = forward_selection_features.copy()
        candidate_features.append(feature)

        # Split the data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(x_imputed[candidate_features], y, test_size=0.2, random_state=42)

        # Fit a model with the selected features
        model = LinearRegression()
        model.fit(X_train, y_train)

        # Evaluate the model with the added feature
        y_pred = model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)

        # Update the best feature if the current one is better (lower MSE)
        if mse < best_feature_mse:
            best_feature_mse = mse
            best_feature = feature

    # Break the loop if change in MSE is smaller than the threshold or if no significant decrease in MSE
    if abs(best_feature_mse - previous_mse) < mse_change_threshold or best_feature is None:
        print("No significant improvement in MSE or no further significant feature found. Stopping forward selection.")
        break

    forward_selection_features.append(best_feature)
    print(f"Added feature: {best_feature}, Mean Squared Error with Features: {best_feature_mse}")

    # Update the previous MSE for the next iteration
    previous_mse = best_feature_mse


Added feature: neck20, Mean Squared Error with Features: 112.90209971511109
Added feature: waist, Mean Squared Error with Features: 110.71544612363748
Added feature: asalw15, Mean Squared Error with Features: 109.73722546106609
Added feature: rawvt_s1, Mean Squared Error with Features: 109.16030776291157
Added feature: smknow15, Mean Squared Error with Features: 108.53770567912167
Added feature: hosnr02, Mean Squared Error with Features: 108.04240044559938
Added feature: shhs1_tcvd, Mean Squared Error with Features: 107.63193302269687
Added feature: age_s1, Mean Squared Error with Features: 107.2211289372306
Added feature: weight, Mean Squared Error with Features: 106.4544484695364
Added feature: pptid, Mean Squared Error with Features: 105.77860099168196
Added feature: diasbp, Mean Squared Error with Features: 105.02303217214656
Added feature: twuweh02, Mean Squared Error with Features: 104.7423698262366
Added feature: nonsp_st, Mean Squared Error with Features: 104.49812557170992
Add

In [ ]:
# Create a new dataset with only the features having non-zero mutual information
forward_selection_dataset = x_imputed[forward_selection_features]

# Add the 'ahi_c0h3a' column to the new dataset
forward_selection_dataset['ahi_c0h3a'] = shhs1['ahi_c0h3a']

<ipython-input-14-132bdb326184>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forward_selection_dataset['ahi_c0h3a'] = shhs1['ahi_c0h3a']


In [ ]:
#write to csv
forward_selection_dataset.to_csv('forward_selection_ahi_c0h3a.csv', index=False)

### Backward selection

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Starting with all the features
backward_selection_features = list(x_imputed.columns)

# Start with an arbitrarily large initial MSE
previous_mse = float('inf')
mse_change_threshold = 1e-4  # Adjust this threshold value based on the scale of your MSE

#run backward selection
while len(backward_selection_features) > 0:
    removed_feature_mse = float('inf')
    feature_to_remove = None

    for feature in backward_selection_features:
        candidate_features = backward_selection_features.copy()
        candidate_features.remove(feature)

        # Split the data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(x_imputed[candidate_features], y, test_size=0.2, random_state=42)

        # Fit a model with the selected features
        model = LinearRegression()
        model.fit(X_train, y_train)

        # Evaluate the model without the current feature
        y_pred = model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)

        # Update the best feature to remove if the current one is better (lower MSE)
        if mse < removed_feature_mse:
            removed_feature_mse = mse
            feature_to_remove = feature

    # Break the loop if change in MSE is smaller than the threshold or if removing a feature increases the MSE
    if abs(removed_feature_mse - previous_mse) < mse_change_threshold or removed_feature_mse > previous_mse:
        print("No significant improvement in MSE or removing any more features deteriorates the model. Stopping backward selection.")
        break

    backward_selection_features.remove(feature_to_remove)
    print(f"Removed feature: {feature_to_remove}, Mean Squared Error without Feature: {removed_feature_mse}")

    # Update the previous MSE for the next iteration
    previous_mse = removed_feature_mse


Removed feature: mh_s1, Mean Squared Error without Feature: 115.55953758172377
Removed feature: o2thpy02, Mean Squared Error without Feature: 111.30286507581754
Removed feature: incar02, Mean Squared Error without Feature: 110.40065693686064
Removed feature: nges02, Mean Squared Error without Feature: 109.66540603375437
Removed feature: weight20, Mean Squared Error without Feature: 109.10009716111378
Removed feature: yrssnr02, Mean Squared Error without Feature: 108.7646633780898
Removed feature: ca15, Mean Squared Error without Feature: 108.30558704616779
Removed feature: sinus15, Mean Squared Error without Feature: 108.0333617883919
Removed feature: thry1, Mean Squared Error without Feature: 107.67478918654383
Removed feature: dias220, Mean Squared Error without Feature: 107.44425234115383
Removed feature: parrptdiab, Mean Squared Error without Feature: 107.1374858858985
Removed feature: sa15, Mean Squared Error without Feature: 106.90384815350156
Removed feature: noise02, Mean Squar

In [ ]:
# Create a new dataset with only the features having non-zero mutual information
backward_selection_dataset = x_imputed[backward_selection_features]

# Add the 'ahi_c0h3a' column to the new dataset
backward_selection_dataset['ahi_c0h3a'] = shhs1['ahi_c0h3a']

<ipython-input-51-d859c8b63163>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  backward_selection_dataset['ahi_c0h3a'] = shhs1['ahi_c0h3a']


In [ ]:
#write to csv
backward_selection_dataset.to_csv('backward_selection_ahi_c0h3a.csv', index=False)

## Target variable: hf15

In [ ]:
y = x_imputed['hf15']
x_imputed = x_imputed.drop(columns=['hf15'])

## Random Forest

In [ ]:
forest = RandomForestRegressor(n_estimators=100)  # You can change the number of trees if needed
forest.fit(x_imputed, y)

RandomForestRegressor()

In [24]:
find_important_features(forest, "random_forest", "hf15")

<ipython-input-18-cfb83178e5c9>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_dataset[target_var] = shhs1[target_var]


277


In [25]:
forest_importances = forest.feature_importances_
random_forest_features = np.array(x_imputed.columns)[forest_importances > 0]
random_forest_dataset = x_imputed[decision_tree_features]
random_forest_dataset['hf15'] = shhs1['hf15']

<ipython-input-25-c550e0a4fed0>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  random_forest_dataset['hf15'] = shhs1['hf15']


In [26]:
len(random_forest_features)

277

In [ ]:
random_forest_dataset.to_csv('random_forest_hf15.csv', index=False)

### Decision Tree

In [27]:
tree = DecisionTreeRegressor()
tree.fit(x_imputed, y)

DecisionTreeRegressor()

In [28]:
find_important_features(tree, "decision_tree", "hf15")

<ipython-input-18-cfb83178e5c9>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_dataset[target_var] = shhs1[target_var]


256


In [29]:
tree_importances = tree.feature_importances_
decision_tree_features = np.array(x_imputed.columns)[tree_importances > 0]
decision_tree_dataset = x_imputed[decision_tree_features]
decision_tree_dataset['hf15'] = shhs1['hf15']

<ipython-input-29-b852afced446>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  decision_tree_dataset['hf15'] = shhs1['hf15']


In [30]:
len(decision_tree_features)

256

In [ ]:
decision_tree_dataset.to_csv('decision_tree_hf15.csv', index=False)

### Mutual Information

In [31]:
mi = mutual_info_regression(x_imputed, y)

In [ ]:
# Get feature names where mutual information is greater than 0
mi_features = np.array(x_imputed.columns)[mi > 0]

# Create a new dataset with only the features having non-zero mutual information
mi_dataset = x_imputed[mi_features]

# Add the 'ahi_c0h3a' column to the new dataset
mi_dataset['hf15'] = shhs1['hf15']

<ipython-input-46-bc022fd210c0>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mi_dataset['hf15'] = shhs1['hf15']


In [ ]:
len(mi_features)

163

In [ ]:
mi_dataset.to_csv('mi_hf15.csv', index=False)

### Forward Selection

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

forward_selection_features = []

# Start with an arbitrarily large initial MSE
previous_mse = float('inf')
mse_change_threshold = 1e-5  # Adjust this threshold value based on the scale of your MSE

while len(forward_selection_features) < len(x_imputed.columns):
    remaining_features = list(set(x_imputed.columns) - set(forward_selection_features))
    best_feature_mse = float('inf')
    best_feature = None

    for feature in remaining_features:
        candidate_features = forward_selection_features.copy()
        candidate_features.append(feature)

        # Split the data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(x_imputed[candidate_features], y, test_size=0.2, random_state=42)

        # Fit a model with the selected features
        model = LinearRegression()
        model.fit(X_train, y_train)

        # Evaluate the model with the added feature
        y_pred = model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)

        # Update the best feature if the current one is better (lower MSE)
        if mse < best_feature_mse:
            best_feature_mse = mse
            best_feature = feature

    # Break the loop if change in MSE is smaller than the threshold or if no significant decrease in MSE
    if abs(best_feature_mse - previous_mse) < mse_change_threshold or best_feature is None:
        print("No significant improvement in MSE or no further significant feature found. Stopping forward selection.")
        break

    forward_selection_features.append(best_feature)
    print(f"Added feature: {best_feature}, Mean Squared Error with Features: {best_feature_mse}")

    # Update the previous MSE for the next iteration
    previous_mse = best_feature_mse

Added feature: ca15, Mean Squared Error with Features: 0.4269018975236238
Added feature: ntg1, Mean Squared Error with Features: 0.4134223021981537
Added feature: paced, Mean Squared Error with Features: 0.40436839177173456
Added feature: anar1a1, Mean Squared Error with Features: 0.39815950724775123
Added feature: nitro15, Mean Squared Error with Features: 0.3922905564772498
Added feature: afib, Mean Squared Error with Features: 0.3878183425443603
Added feature: emphys15, Mean Squared Error with Features: 0.3844633249093897
Added feature: sob02, Mean Squared Error with Features: 0.3822874616675461
Added feature: mi15, Mean Squared Error with Features: 0.3804759498649544
Added feature: sa15, Mean Squared Error with Features: 0.37902380769269334
Added feature: pacem15, Mean Squared Error with Features: 0.3776377614790319
Added feature: yrsns15, Mean Squared Error with Features: 0.3767714850356414
Added feature: diasbp, Mean Squared Error with Features: 0.3760827810924168
Added feature: 

In [ ]:
len(forward_selection_features)

93

In [ ]:
# Create a new dataset with only the features having non-zero mutual information
forward_selection_dataset = x_imputed[forward_selection_features]

# Add the 'ahi_c0h3a' column to the new dataset
forward_selection_dataset['hf15'] = shhs1['hf15']

<ipython-input-36-c416a05ffb5d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forward_selection_dataset['hf15'] = shhs1['hf15']


In [ ]:
forward_selection_dataset.to_csv('forward_selection_hf15.csv', index=False)

### Backward selection

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Starting with all the features
backward_selection_features = list(x_imputed.columns)

# Start with an arbitrarily large initial MSE
previous_mse = float('inf')
mse_change_threshold = 1e-4  # Adjust this threshold value based on the scale of your MSE

while len(backward_selection_features) > 0:
    removed_feature_mse = float('inf')
    feature_to_remove = None

    for feature in backward_selection_features:
        candidate_features = backward_selection_features.copy()
        candidate_features.remove(feature)

        # Split the data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(x_imputed[candidate_features], y, test_size=0.2, random_state=42)

        # Fit a model with the selected features
        model = LinearRegression()
        model.fit(X_train, y_train)

        # Evaluate the model without the current feature
        y_pred = model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)

        # Update the best feature to remove if the current one is better (lower MSE)
        if mse < removed_feature_mse:
            removed_feature_mse = mse
            feature_to_remove = feature

    # Break the loop if change in MSE is smaller than the threshold or if removing a feature increases the MSE
    if abs(removed_feature_mse - previous_mse) < mse_change_threshold or removed_feature_mse > previous_mse:
        print("No significant improvement in MSE or removing any more features deteriorates the model. Stopping backward selection.")
        break

    backward_selection_features.remove(feature_to_remove)
    print(f"Removed feature: {feature_to_remove}, Mean Squared Error without Feature: {removed_feature_mse}")

    # Update the previous MSE for the next iteration
    previous_mse = removed_feature_mse

Removed feature: mh_s1, Mean Squared Error without Feature: 0.5081974546928006
Removed feature: rawrp_s1, Mean Squared Error without Feature: 0.4272055561900398
Removed feature: anar31, Mean Squared Error without Feature: 0.4208104287967728
Removed feature: prev_hx_mi, Mean Squared Error without Feature: 0.41741884266835055
Removed feature: angina15, Mean Squared Error without Feature: 0.4148126211891616
Removed feature: asthma15, Mean Squared Error without Feature: 0.4124365154472988
Removed feature: othrcs15, Mean Squared Error without Feature: 0.410051986768463
Removed feature: pcs_s1, Mean Squared Error without Feature: 0.40776657207185946
Removed feature: lvhst, Mean Squared Error without Feature: 0.4057659438974485
Removed feature: rawbp_s1, Mean Squared Error without Feature: 0.40381827762617506
Removed feature: nges02, Mean Squared Error without Feature: 0.40198607562182714
Removed feature: apbs, Mean Squared Error without Feature: 0.40013911561110027
Removed feature: beta1, Me

In [ ]:
# Create a new dataset with only the features having non-zero mutual information
backward_selection_dataset = x_imputed[backward_selection_features]

# Add the 'ahi_c0h3a' column to the new dataset
backward_selection_dataset['hf15'] = shhs1['hf15']

<ipython-input-55-be33fb0c6a21>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  backward_selection_dataset['hf15'] = shhs1['hf15']


In [ ]:
len(backward_selection_features)

209

In [ ]:
backward_selection_dataset.to_csv('backward_selection_hf15.csv', index=False)